In [169]:
from PIL import Image, ImageEnhance
import numpy as np 
import os
import cv2
from skimage.color import rgb2lab, lab2rgb
from skimage.io import imsave
# np.save('example_1', a)
# b = np.load('example/example_1.npy')
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose
from keras.layers import Activation, Dense, Dropout, Flatten
#from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
#from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import random
import tensorflow as tf
from keras.models import load_model
import keras

In [180]:
from sklearn.ensemble import RandomForestRegressor

In [170]:
# image_folder = "/Users/alexeyvaganov/doc/files/color_matcher/type"
image_massiv = "/Users/alexeyvaganov/doc/files/color_matcher/temp"

In [205]:
pixel = 1

# Обучение

In [172]:
def crop_image(array_x_y,a,b):
    h=0
    while h <= a.shape[0]-pixel:
        w=0
        while w <= a.shape[1]-pixel:
            #print (h,",",h+56)
            x_ = a[h:h+pixel,w:w+pixel]
            y_ = b[h:h+pixel,w:w+pixel]
            array_x_y.append([x_,y_])
            w+=pixel
        h+=pixel

In [173]:
def show_image(img_file):
    img = Image.open(img_file)
    img.show()

array_transform_nastr = [
    [1.5,0.5,0.5],
    [1.5,1,0.5],
    [1,1,0.8],
    [1,1,2],
    [2,1.2,0],
    [2,1.2,0.1]
]

In [174]:
def transform_image(img_pil,brightness,contrast,color):
    if (brightness != 0):
        enhancer = ImageEnhance.Brightness(img_pil)
        img_pil = enhancer.enhance(brightness)
    if (contrast != 0):
        enhancer = ImageEnhance.Contrast(img_pil)
        img_pil = enhancer.enhance(contrast)
    if (color != 0):
        enhancer = ImageEnhance.Color(img_pil)
        img_pil = enhancer.enhance(color) 
    return img_pil

# Из нескольких папок по разным фото

In [145]:
image_folder = "/Users/alexeyvaganov/doc/files/color_matcher/type"
array_image = []
for root, dirs, files in os.walk(image_folder):
    for dir_ in dirs:
        #print (dir_)
        for root_, dirs_, files_ in os.walk(image_folder + "/" + dir_):
            for file in files_:
                if file == ".DS_Store" or not file[0] == "_" or file[1] == "Y":
                    continue
                if file[1] == "X":
                    #print(file)
                    img_X = Image.open(root_ + "/" + file)
                    img_X = img_X.convert('RGB')
                    img_X_lab = rgb2lab(img_X)
                    
                    img_Y = Image.open(root_ + "/_Y.jpeg")
                    img_Y = img_Y.convert('RGB')
                    img_Y_lab = rgb2lab(img_Y)

                    array_image.append([np.array(img_X_lab),np.array(img_Y_lab)])
                    
array_full_image = np.array(array_image) 
array_full_image.shape

/var/folders/87/c02w9bp14k1fx5943h90k0500000gn/T/ipykernel_87464/1504464291.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  array_full_image = np.array(array_image)


(25, 2)

# Из одной папки

In [206]:
image_folder = "/Users/alexeyvaganov/doc/files/color_matcher/slide_X"
array_image = []
for root, dirs, files in os.walk(image_folder):
    for file in files:
        if file == ".DS_Store":
            continue
        img_pil = Image.open(root + "/" + file)
        img_pil = img_pil.convert('RGB')
        img_pil_lab = rgb2lab(img_pil)
        #img_cv2_ish = cv2.imread(root + "/" + file)
        for x in array_transform_nastr:
            img_ = transform_image(img_pil,x[0],x[1],x[2])
            #img_.save("/Users/alexeyvaganov/doc/files/color_matcher/temp/geeks.jpg")
            #img_cv2_transf = cv2.imread("/Users/alexeyvaganov/doc/files/color_matcher/temp/geeks.jpg")
            img_lab = rgb2lab(img_)
            #img_pil = rgb2lab(img_pil)
            array_image.append([np.array(img_lab),np.array(img_pil_lab)])
        
array_full_image = np.array(array_image) 
array_full_image.shape

(6, 2, 1080, 864, 3)

In [220]:
array_full_image[0][0][0][0]

array([ 8.81154370e+01, -2.20342180e-03,  4.17666359e-03])

# Модель 1

In [165]:
# Building the neural network
model = Sequential()
model.add(InputLayer(input_shape=(None, None, 2)))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
#model.compile(optimizer='rmsprop', loss='mse', metrics=[keras.metrics.SparseCategoricalAccuracy()])
#model.compile(optimizer='rmsprop', loss='mse')


In [166]:
model.compile(optimizer='adam', loss='mse',metrics=['accuracy'])

# Модель 2

In [56]:
model = Sequential()
model.add(InputLayer(input_shape=(56, 56, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))


In [57]:
#model.compile(optimizer='rmsprop', loss='mse')
model.compile(optimizer='adam', loss='mse',metrics=['accuracy'])

# Подготовка данных

In [207]:
array_x_y = []
i=0
x_sgape = array_full_image.shape[0]
while i < x_sgape:
    crop_image(array_x_y,array_full_image[i,0],array_full_image[i,1])
    i+=1
array_crop_image = np.array(array_x_y) 
array_crop_image = 1.0/255*array_crop_image
array_crop_image.shape

(5598720, 2, 1, 1, 3)

In [208]:
X_ = array_crop_image[:,0]
Y_ = array_crop_image[:,1]
X_ = X_[:,:,:,1:]
Y_ = Y_[:,:,:,1:]

# Обучение нейросеть

In [167]:
model.fit(X_,Y_,batch_size=32, epochs=20, validation_split=0.2)

Epoch 1/20


2022-06-01 18:25:41.014183: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


174/174 [==============================] - ETA: 0s - loss: 0.0034 - accuracy: 0.7271

2022-06-01 18:25:43.470276: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


174/174 [==============================] - 3s 15ms/step - loss: 0.0034 - accuracy: 0.7271 - val_loss: 0.0038 - val_accuracy: 0.7363
Epoch 2/20
174/174 [==============================] - 2s 13ms/step - loss: 0.0030 - accuracy: 0.7597 - val_loss: 0.0038 - val_accuracy: 0.7204
Epoch 3/20
174/174 [==============================] - 2s 13ms/step - loss: 0.0029 - accuracy: 0.7621 - val_loss: 0.0037 - val_accuracy: 0.7300
Epoch 4/20
174/174 [==============================] - 2s 13ms/step - loss: 0.0029 - accuracy: 0.7640 - val_loss: 0.0040 - val_accuracy: 0.7230
Epoch 5/20
174/174 [==============================] - 2s 14ms/step - loss: 0.0028 - accuracy: 0.7630 - val_loss: 0.0037 - val_accuracy: 0.7338
Epoch 6/20
174/174 [==============================] - 2s 13ms/step - loss: 0.0028 - accuracy: 0.7687 - val_loss: 0.0038 - val_accuracy: 0.7073
Epoch 7/20
174/174 [==============================] - 2s 13ms/step - loss: 0.0028 - accuracy: 0.7669 - val_loss: 0.0042 - val_accuracy: 0.7028
Epoch 8/20

In [111]:
model = load_model("/Users/alexeyvaganov/doc/files/color_matcher/temp/type 38IMG_2837.h5")

# Регрессия

In [181]:
model = RandomForestRegressor()

In [227]:
X_train = X_[:100]
Y_train = Y_[:100]

In [249]:
test_ = X_train[:]

In [250]:
test_.shape

(100, 1, 1, 2)

In [244]:
Y_.shape

(5598720, 1, 1, 2)

In [229]:
model.fit(X_train, Y_train)

ValueError: Found array with dim 4. RandomForestRegressor expected <= 2.

# Предсказание

In [11]:
Obr = 62

In [151]:
file_name_massiv = []
file_name_massiv.append("IMG_2825.jpeg")
file_name_massiv.append("IMG_3379.jpeg")
file_name_massiv.append("IMG_2915.jpeg")
file_name_massiv.append("IMG_2980.jpeg")
file_name_massiv.append("IMG_3107.jpeg")
file_name_massiv.append("IMG_3323.jpeg")
file_name_massiv.append("IMG_3336.jpeg")
file_name_massiv.append("IMG_2825.jpeg")
file_name_massiv.append("IMG_2769.jpeg")
file_name_massiv.append("IMG_2837.jpeg")
file_name_massiv.append("IMG_2847.jpeg")

In [152]:
def add_testing_file(file_name):
    test_image_path = image_massiv + "/" + file_name
    img_test = Image.open(test_image_path)
    img_test = img_test.convert('RGB')
    #array_img_test.append(img_test) 
    img_test = rgb2lab(img_test)
    #array_full_img_test = np.array(img_test) 
    return np.array(img_test) 
    #array_full_img_test.shape
    

In [153]:
def predict_image(array_full_img_test):
    new_img = np.zeros((array_full_img_test.shape[0], array_full_img_test.shape[1], 3))
    h=0
    while h <= array_full_img_test.shape[0]-pixel:
        w=0
        while w <= array_full_img_test.shape[1]-pixel:
            # predict
            x_iz = array_full_img_test[h:h+pixel,w:w+pixel]
            x_ = x_iz[:,:,1:]
            x_ = 1.0/255*x_
            x_ = x_.reshape(1,pixel,pixel,2)
            pred_y = model.predict(x_)

            cur = np.zeros((pixel, pixel, 3))
            cur[:,:,0] = x_iz[:,:,0]
            pred_y = pred_y[0]* 256
            cur[:,:,1:] = pred_y

            new_img[h:h+pixel,w:w+pixel] = cur
            w+=pixel
        h+=pixel 
    return new_img

In [154]:
arrive_testing_file = []
for name_test_file in file_name_massiv:
    arr_ = add_testing_file(name_test_file)
    arrive_testing_file.append([name_test_file, arr_])

# Предсказание здесь

In [168]:
Obr += 1
model_name = image_massiv + "/" + str(Obr) + "_type" + ".h5"
print(model_name)
for m_image in arrive_testing_file:
    p_image = predict_image(m_image[1])
    image_name = image_massiv + "/" + str(Obr) + "_" + m_image[0]
    print(image_name)
    imsave(image_name, lab2rgb(p_image))
model.save(model_name)
    

/Users/alexeyvaganov/doc/files/color_matcher/temp/68_type.h5
1/1 [==============================] - 0s 10ms/step


2022-06-01 18:26:35.757930: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 11ms/step


1/1 [==============================] - 0s 9ms/step
/Users/alexeyvaganov/doc/files/color_matcher/temp/68_IMG_2825.jpeg


Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


...

1/1 [==============================] - 0s 9ms/step
/Users/alexeyvaganov/doc/files/color_matcher/temp/68_IMG_2837.jpeg


Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


1/1 [==============================] - 0s 8ms/step


1/1 [==============================] - 0s 9ms/step
/Users/alexeyvaganov/doc/files/color_matcher/temp/68_IMG_2847.jpeg


Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


# Старые

In [81]:
file_name = "IMG_2825.jpeg"
test_image_path = image_massiv + "/" + file_name
img_test = Image.open(test_image_path)
img_test = img_test.convert('RGB')
#array_img_test.append(img_test) 
img_test = rgb2lab(img_test)
array_full_img_test = np.array(img_test) 
array_full_img_test.shape

(1280, 960, 3)

In [88]:
Obr += 1

new_img = np.zeros((array_full_img_test.shape[0], array_full_img_test.shape[1], 3))
h=0
while h <= array_full_img_test.shape[0]-pixel:
    w=0
    while w <= array_full_img_test.shape[1]-pixel:
        # predict
        x_iz = array_full_img_test[h:h+pixel,w:w+pixel]
        x_ = x_iz[:,:,1:]
        x_ = 1.0/255*x_
        x_ = x_.reshape(1,pixel,pixel,2)
        pred_y = model.predict(x_)
        
        cur = np.zeros((pixel, pixel, 3))
        cur[:,:,0] = x_iz[:,:,0]
        pred_y = pred_y[0]* 256
        cur[:,:,1:] = pred_y
        
        new_img[h:h+pixel,w:w+pixel] = cur
        
        w+=pixel
    h+=pixel
    
image_name = image_massiv + "/" + str(Obr) + file_name
model_name = image_massiv + "/" + "type " + str(Obr) + file_name + ".h5"

imsave(image_name, lab2rgb(new_img))
model.save(model_name)


1/1 [==============================] - 0s 10ms/step


1/1 [==============================] - 0s 10ms/step


Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


In [46]:
model.save(image_massiv + "/" + str(Obr) + "type_.h5")

In [21]:
model = load_model("/Users/alexeyvaganov/doc/files/color_matcher/temp/8type_.h5")